In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

In [2]:
df = pd.read_csv("fulldataset.csv")

In [3]:
df

,Unnamed: 0,Unnamed: 0.1,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,usCite_1,...,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,opiniontext
0,0,321,1812-033,1812-033-01,1812-033-01-01,1812-033-01-01-01,3/14/1812,1,11 U.S. 32,11.0,...,7.0,1.0,120.0,NaN,15.0,14.0,1,7,0,<p><strong>United States v. Hudson</strong></p...
1,1,427,1815-006,1815-006-01,1815-006-01-01,1815-006-01-01-01,2/17/1815,1,13 U.S. 43,13.0,...,2.0,8.0,800.0,NaN,19.0,14.0,1,5,0,<p><strong>Terrett v. Taylor</strong></p><p><s...
2,2,453,1815-032,1815-032-01,1815-032-01-01,1815-032-01-01-01,3/10/1815,1,13 U.S. 292,13.0,...,NaN,9.0,900.0,NaN,19.0,14.0,1,7,0,<p><strong>Town of Pawlet v. Clark</strong></p...
3,3,769,1824-023,1824-023-01,1824-023-01-01,1824-023-01-01-01,3/11/1824,1,22 U.S. 445,22.0,...,NaN,9.0,900.0,NaN,19.0,14.0,1,6,0,<p><strong>Mason v. Muncaster</strong></p><p><...
4,4,1677,1845-032,1845-032-01,1845-032-01-01,1845-032-01-01-01,2/11/1845,1,44 U.S. 266,44.0,...,NaN,6.0,600.0,NaN,29.0,25.0,1,7,0,<p><strong>White v. Nicholls</strong></p><p><s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,1269,13082,2013-068,2013-068-01,2013-068-01-02,2013-068-01-02-01,6/26/14,1,573 U.S. 464,573.0,...,4.0,8.0,800.0,NaN,NaN,111.0,1,9,0,"<p style=""p-NOTICEOpinions"">NOTICE: This opini..."
793,1270,13084,2013-070,2013-070-01,2013-070-01-01,2013-070-01-01-01,6/30/14,1,573 U.S. 682,573.0,...,NaN,3.0,361.0,NaN,112.0,111.0,1,5,4,"<p style=""p-NOTICEOpinions"">NOTICE: This opini..."
794,1272,13089,2013-073,2013-073-01,2013-073-01-01,2013-073-01-01-01,6/19/14,1,573 U.S. 228,573.0,...,7.0,2.0,200.0,NaN,113.0,111.0,1,9,0,"<p style=""p-NOTICEOpinions"">NOTICE: This opini..."
795,1273,13093,2014-003,2014-003-01,2014-003-01-01,2014-003-01-01-01,1/20/15,1,574 U.S. 352,574.0,...,NaN,6.0,600.0,Religious Land Use & Institutionalized Persons...,112.0,111.0,1,9,0,<p>NOTICE: This opinion is subject to formal r...


In [4]:
df.loc[545:610]

,Unnamed: 0,Unnamed: 0.1,caseId,docketId,caseIssuesId,voteId,dateDecision,decisionType,usCite,usCite_1,...,authorityDecision2,lawType,lawSupp,lawMinor,majOpinWriter,majOpinAssigner,splitVote,majVotes,minVotes,opiniontext
545,792,7833,1980-142,1980-142-01,1980-142-01-01,1980-142-01-01-01,6/29/81,1,453 U.S. 280,453.0,...,4.0,6.0,600.0,22 U.S.C. § 211,99.0,99.0,1,7,2,<p><strong>Haig v. Agee</strong></p><p><strong...
546,794,7848,1980-149,1980-149-01,1980-149-01-01,1980-149-01-01-01,7/2/81,7,453 U.S. 490,453.0,...,NaN,2.0,200.0,NaN,95.0,92.0,1,6,3,"<p><strong>Metromedia, Inc. v. City of San Die..."
547,795,7858,1981-004,1981-004-01,1981-004-01-01,1981-004-01-01-01,11/10/81,1,454 U.S. 27,454.0,...,NaN,3.0,330.0,NaN,95.0,99.0,1,9,0,<p><strong>FEC v. Democratic Senatorial Campai...
548,797,7862,1981-007,1981-007-01,1981-007-01-01,1981-007-01-01-01,11/30/81,2,454 U.S. 90,454.0,...,NaN,2.0,200.0,NaN,NaN,99.0,1,6,2,<p><strong>California ex rel. Cooper v. Mitche...
549,798,7872,1981-014,1981-014-01,1981-014-01-01,1981-014-01-01-01,12/8/81,1,454 U.S. 263,454.0,...,NaN,2.0,203.0,NaN,101.0,99.0,1,8,1,<p><strong>Widmar v. Vincent</strong></p><p><s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,886,9019,1985-095,1985-095-01,1985-095-01-01,1985-095-01-01-01,6/2/86,1,476 U.S. 488,476.0,...,NaN,2.0,200.0,NaN,102.0,99.0,1,9,0,<p><strong>City of Los Angeles v. Preferred Co...
607,887,9030,1985-104,1985-104-01,1985-104-01-01,1985-104-01-01-01,6/11/86,1,476 U.S. 693,476.0,...,NaN,2.0,202.0,NaN,99.0,99.0,1,8,1,<p><strong>Bowen v. Roy</strong></p><p><strong...
608,889,9062,1985-125,1985-125-01,1985-125-01-01,1985-125-01-01-01,6/25/86,1,477 U.S. 242,477.0,...,NaN,4.0,400.0,NaN,95.0,95.0,1,6,3,"<p><strong>Anderson v. Liberty Lobby, Inc.</st..."
609,890,9083,1985-141,1985-141-01,1985-141-01-01,1985-141-01-01-01,6/30/86,1,478 U.S. 1,478.0,...,NaN,2.0,200.0,NaN,99.0,99.0,1,7,2,<p><strong>Press-Enterprise Co. v. Superior Co...


### Run the three cells below:

In [5]:
#up to 535, we have MR. JUSTICE, and then 536 and after it's just JUSTICE

s = df.iloc[534]['opiniontext']
soup = BeautifulSoup(s, 'html.parser')
soup

# p > "MR. JUSTICE" and "DELIVERED"


<p><strong>Central Hudson Gas &amp; Electric Corp. v.</strong></p><p><strong>Public Service Commission of New York</strong></p><p><strong>No. 79-565</strong></p><p><strong>Argued March 17, 1980</strong></p><p><strong>Decided June 20, 1980</strong></p><p><strong></strong></p><p><em>APPEAL FROM THE COURT OF APPEALS OF NEW YORK
</em></p><p><em><em>Syllabus</em> </em></p><p></p><p>47 N.Y.2d 94, 390 N.E.2d 749, reversed.</p><p>MR. JUSTICE POWELL delivered the opinion of the Court.</p><p>This case presents the question whether a regulation of the Public Service Commission of the State of New York violates the First and Fourteenth Amendments because it completely bans promotional advertising by an electrical utility.</p><p><strong class="heading-5 font-w-bold">I</strong></p><p>In December, 1973, the Commission, appellee here, ordered electric utilities in New York State to cease all advertising that "promot[es] the use of electricity."  App. to Juris.Statement </p><p> 31a.  The order was base

In [6]:
ops = []
for i in soup.strings:
    if repr(i).find("MR. JUSTICE") == 1:
        ops.append(i)
ops

['MR. JUSTICE POWELL delivered the opinion of the Court.',
 'MR. JUSTICE BRENNAN, concurring in the judgment.',
 'MR. JUSTICE BLACKMUN, with whom MR. JUSTICE BRENNAN joins, concurring in the judgment.',
 'MR. JUSTICE STEVENS, with whom MR. JUSTICE BRENNAN joins, concurring in the judgment.',
 'MR. JUSTICE REHNQUIST, dissenting.']

In [7]:
# SPLITTING THE OPINIONS BY OPINION
# https://stackoverflow.com/a/13184791/14336451 <- regex multiple delimiters (like a list)
# FOR KEEPING THE DELIMITERS INTACT: regexPattern = '|'.join('(?<={})'.format(re.escape(delim)) for delim in ops)

regexPattern = '|'.join(map(re.escape, ops))
l = re.split(regexPattern, df.iloc[534]['opiniontext'])
l.pop(0)
l

['</p><p>This case presents the question whether a regulation of the Public Service Commission of the State of New York violates the First and Fourteenth Amendments because it completely bans promotional advertising by an electrical utility.</p><p><strong class="heading-5 font-w-bold">I</strong></p><p>In December, 1973, the Commission, appellee here, ordered electric utilities in New York State to cease all advertising that "promot[es] the use of electricity."  App. to Juris.Statement </p><p> 31a.  The order was based on the Commission\'s finding that "the interconnected utility system in New York State does not have sufficient fuel stocks or sources of supply to continue furnishing all customer demands for the 1973-1974 winter."  <em>Id.</em> at 26a.</p><p>Three years later, when the fuel shortage had eased, the Commission requested comments from the public on its proposal to continue the ban on promotional advertising.  Central Hudson Gas &amp; Electric Corp., the appellant in this c

Ignore the below until the next markdown cell

In [5]:
regexPattern = '|'.join(map(re.escape, ops))
regexPattern

'MR\\.\\ JUSTICE\\ POWELL\\ delivered\\ the\\ opinion\\ of\\ the\\ Court\\.|MR\\.\\ JUSTICE\\ BRENNAN,\\ concurring\\ in\\ the\\ judgment\\.|MR\\.\\ JUSTICE\\ BLACKMUN,\\ with\\ whom\\ MR\\.\\ JUSTICE\\ BRENNAN\\ joins,\\ concurring\\ in\\ the\\ judgment\\.|MR\\.\\ JUSTICE\\ STEVENS,\\ with\\ whom\\ MR\\.\\ JUSTICE\\ BRENNAN\\ joins,\\ concurring\\ in\\ the\\ judgment\\.|MR\\.\\ JUSTICE\\ REHNQUIST,\\ dissenting\\.'

In [ ]:
for i in range(535):
    s = df.iloc[i]['opiniontext']
    soup = BeautifulSoup(s, 'html.parser')
    ops = []
    for i in soup.strings:
        if repr(i).find("MR. JUSTICE") == 1:
            ops.append(i)
    regexPattern = '|'.join(map(re.escape, ops))
    re.split(regexPattern, df.iloc[i]['opiniontext'])

In [ ]:
for i in [536:]:
    s = df.iloc[i]['opiniontext']
    soup = BeautifulSoup(s, 'html.parser')
    ops = []
    for i in soup.strings:
        if repr(i).find("JUSTICE") == 1 or repr(i).find("THE CHIEF JUSTICE") == 1:
            ops.append(i)
    regexPattern = '|'.join(map(re.escape, ops))
    re.split(regexPattern, df.iloc[i]['opiniontext'])

In [116]:
s1 = df.iloc[536]['opiniontext']
soup2 = BeautifulSoup(s1, 'html.parser')
for i in soup2.strings:
    if repr(i).find("JUSTICE") == 1 or repr(i).find("THE CHIEF JUSTICE") == 1:
        print(i)

THE CHIEF JUSTICE and JUSTICE BLACKMUN dissent.  They would grant certiorari and give this case plenary consideration.
JUSTICE STEWART dissents from this summary reversal of the courts of Kentucky, which, so far as appears, applied wholly correct constitutional criteria in reaching their decisions.
JUSTICE REHNQUIST, dissenting.


In [110]:
df.iloc[536]['opiniontext']

'<p><strong>Stone v. Graham</strong></p><p><strong>No. 80-321</strong></p><p><strong>Decided November 17, 1980</strong></p><p><strong></strong></p><p><em>ON PETITION FOR WRIT OF CERTIORARI TO THE\r</em></p><p><em>SUPREME COURT OF KENTUCKY\r</em></p><p><em><em>Syllabus</em> </em></p><p><em>Held:</em>  A Kentucky statute requiring the posting of a copy of the Ten Commandments, purchased with private contributions, on the wall of each public school classroom in the State has no secular legislative purpose, and therefore is unconstitutional as violating the Establishment Clause of the First Amendment.  While the state legislature required the notation in small print at the bottom of each display that</p><p>"[t]he secular application of the Ten Commandments is clearly seen in its adoption as the fundamental legal code of Western Civilization and the Common Law of the United States,"</p><p>PER CURIAM.</p><p>This Court has announced a three-part test for determining whether a challenged state

In [120]:
s2 = df.iloc[710]['opiniontext']
soup3 = BeautifulSoup(s2, 'html.parser')
for i in soup3.strings:
    if repr(i).find("JUSTICE") == 1 or repr(i).find("THE CHIEF JUSTICE") == 1:
        print(i)

JUSTICE STEVENS delivered the opinion of the Court.
JUSTICE SOUTER, with whom THE CHIEF JUSTICE and JUSTICE SCALIA join, and with whom JUSTICE THOMAS joins except as to Part II, dissenting.
JUSTICE THOMAS, with whom JUSTICE SCALIA joins as to Part II, dissenting.


In [118]:
df.iloc[700]['opiniontext']

'<p>OCTOBER TERM, 1994</p><p>Syllabus</p><p>HURLEY ET AL. <em>v. </em>IRISH-AMERICAN GAY, LESBIAN AND BISEXUAL GROUP</p><p>OF BOSTON, INC., ET AL.</p><p>CERTIORARI TO THE SUPREME JUDICIAL COURT OF MASSACHUSETTS</p><p>\nNo. 94-749. Argued April 25, 1995-Decided June 19, 1995</p><p>Petitioner South Boston Allied War Veterans Council, an unincorporated association of individuals elected from various veterans groups, was authorized by the city of Boston to organize and conduct the St. Patrick\'s Day-Evacuation Day Parade. The Council refused a place in the 1993 event to respondent GLIB, an organization formed for the purpose of marching in the parade in order to express its members\' pride in their Irish heritage as openly gay, lesbian, and bisexual individuals, to show that there are such individuals in the community, and to support the like men and women who sought to march in the New York St. Patrick\'s Day parade. GLIB and some of its members filed this suit in state court, alleging th

In [148]:
for i in range(5):
    print(i)

0
1
2
3
4


### Here's where the regex pattern gets messed all the way up

In [8]:
testdict = {}
for i in range(2):
# initialize empty opinions dictionary
    testops = {}
# begin opinion text processing
    s = df.iloc[i]['opiniontext']
    soup = BeautifulSoup(s, 'html.parser')
# initialize first opinions array for splitting and categoring
    ops = []
# split opinions based on justice declaration and add to opinions array
    for j in soup.strings:
        if repr(j).find("MR. JUSTICE") == 1:
            ops.append(j)
    print(ops)
# build regex pattern
    regexPattern = '|'.join(map(re.escape, ops))
    print(regexPattern)
    l = re.split(regexPattern, df.iloc[i]['opiniontext'])
# remove first value of split opinions array because it's not an opinion, it's a summary
    l.pop(0)
# initialize second opinion array for categorizing as "majority", "concurring", or "dissenting"
    op = []
    for m in ops:
        if m.find("delivered the opinion") > 1:
            op.append("Majority")
        if m.find("concurring") > 1:
            op.append("Concurring")
        if m.find("dissent") > 1:
            op.append("Dissenting")
# the else below is to catch errors—we should test the above first
        else:
            op.append("Double check content")
# add the category as key and the opinion text as value to the dictionary "opinions"
    for k in range(len(ops)):
        testops[op[k]] = l[k]
        print(testops)
# add the case id as key and the opinions dictionary as value to the dictionary "clean_opinions_json"
    testdict[df.iloc[i]['caseId']] = testops

[]

[]



Below are just tests

In [9]:
testops = {}
for k in range(len(ops)):
    testops[ops[k]] = 'value'

In [10]:
testops

{}

In [23]:
op = []

In [11]:
s = df.iloc[534]['opiniontext']
soup = BeautifulSoup(s, 'html.parser')

op = []
ops = []
for i in soup.strings:
    if repr(i).find("MR. JUSTICE") == 1:
        ops.append(i)

for m in ops:
    if m.find("delivered the opinion") > 1:
        op.append("Majority")
    elif m.find("concurring") > 1:
        op.append("Concurring")
    elif m.find("dissent") > 1:
        op.append("Dissenting")
op

['Majority', 'Concurring', 'Concurring', 'Concurring', 'Dissenting']

In [12]:
ops

['MR. JUSTICE POWELL delivered the opinion of the Court.',
 'MR. JUSTICE BRENNAN, concurring in the judgment.',
 'MR. JUSTICE BLACKMUN, with whom MR. JUSTICE BRENNAN joins, concurring in the judgment.',
 'MR. JUSTICE STEVENS, with whom MR. JUSTICE BRENNAN joins, concurring in the judgment.',
 'MR. JUSTICE REHNQUIST, dissenting.']

In [13]:
for m in ops:
    if m.find("delivered the opinion"):
        print(m)

MR. JUSTICE POWELL delivered the opinion of the Court.
MR. JUSTICE BRENNAN, concurring in the judgment.
MR. JUSTICE BLACKMUN, with whom MR. JUSTICE BRENNAN joins, concurring in the judgment.
MR. JUSTICE STEVENS, with whom MR. JUSTICE BRENNAN joins, concurring in the judgment.
MR. JUSTICE REHNQUIST, dissenting.


In [38]:
for i in range(2):
    s = df.iloc[i]['opiniontext']
    soup = BeautifulSoup(s, 'html.parser')
    ops = []
    for j in soup.strings:
        if repr(j).find("MR. JUSTICE") == 1:
            ops.append(j)
    regexPattern = '|'.join(map(re.escape, ops))
    opinions = re.split(regexPattern, df.iloc[i]['opiniontext'])
opinions

['',
 '<',
 'p',
 '>',
 '<',
 's',
 't',
 'r',
 'o',
 'n',
 'g',
 '>',
 'T',
 'e',
 'r',
 'r',
 'e',
 't',
 't',
 ' ',
 'v',
 '.',
 ' ',
 'T',
 'a',
 'y',
 'l',
 'o',
 'r',
 '<',
 '/',
 's',
 't',
 'r',
 'o',
 'n',
 'g',
 '>',
 '<',
 '/',
 'p',
 '>',
 '<',
 'p',
 '>',
 '<',
 's',
 't',
 'r',
 'o',
 'n',
 'g',
 '>',
 '1',
 '3',
 ' ',
 'U',
 '.',
 'S',
 '.',
 ' ',
 '(',
 '9',
 ' ',
 'C',
 'r',
 'a',
 'n',
 'c',
 'h',
 ')',
 ' ',
 '4',
 '3',
 '<',
 '/',
 's',
 't',
 'r',
 'o',
 'n',
 'g',
 '>',
 '<',
 '/',
 'p',
 '>',
 '<',
 'p',
 '>',
 '<',
 's',
 't',
 'r',
 'o',
 'n',
 'g',
 '>',
 '<',
 '/',
 's',
 't',
 'r',
 'o',
 'n',
 'g',
 '>',
 '<',
 '/',
 'p',
 '>',
 '<',
 'p',
 '>',
 '<',
 'e',
 'm',
 '>',
 'E',
 'R',
 'R',
 'O',
 'R',
 ' ',
 'T',
 'O',
 ' ',
 'T',
 'H',
 'E',
 ' ',
 'C',
 'I',
 'R',
 'C',
 'U',
 'I',
 'T',
 ' ',
 'C',
 'O',
 'U',
 'R',
 'T',
 ' ',
 'O',
 'F',
 ' ',
 'T',
 'H',
 'E',
 ' ',
 'C',
 'O',
 'U',
 'N',
 'T',
 'Y',
 '\r',
 '<',
 '/',
 'e',
 'm',
 '>',
 '<',
 '/',
 'p'

In [ ]:
    regexPattern = '|'.join(map(re.escape, ops))
    array = re.split(regexPattern, df.iloc[i]['opiniontext'])

### Final code setup

In [14]:

# total dictionary outside of loop/cell
clean_opinions_json = {}

In [16]:
for i in range(536):
# initialize empty opinions dictionary
    opinions = {}
# begin opinion text processing
    s = df.iloc[i]['opiniontext']
    soup = BeautifulSoup(s, 'html.parser')
# initialize first opinions array for splitting and categoring
    ops = []
# split opinions based on justice declaration and add to opinions array
    for j in soup.strings:
        if repr(j).find("MR. JUSTICE") == 1:
            ops.append(j)
# build regex pattern
    regexPattern = '|'.join(map(re.escape, ops))
    array = re.split(regexPattern, df.iloc[i]['opiniontext'])
# remove first value of split opinions array because it's not an opinion, it's a summary
    array.pop(0)
# initialize second opinion array for categorizing as "majority", "concurring", or "dissenting"
    op = []
    for m in ops:
        if m.find("delivered"):
            op.append("Majority")
        elif m.find("concurring"):
            op.append("Concurring")
        elif m.find("dissent"):
            op.append("Dissenting")
# the else below is to catch errors—we should test the above first
       else:
            op.append("Double check content")
# add the category as key and the opinion text as value to the dictionary "opinions"
    for k in range(len(ops)):
        opinions[op[k]] = array[k]
# add the case id as key and the opinions dictionary as value to the dictionary "clean_opinions_json"
    clean_opinions_json[df.iloc[i]['caseId']] = opinions

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 28)